In [1]:
import pandas as pd
import cobra
from cobra.io import load_json_model
from cobra.flux_analysis import single_gene_deletion, single_reaction_deletion, double_gene_deletion
import numpy as np

import re
from glob import glob
from Bio import Entrez, SeqIO

# I. Gene comparison EcN aerobic vs. anaerobic

In [203]:
# Load EcN model
EcN_ID = 'CP022686.1'
EcN_model = cobra.io.load_json_model('../data/models/%s_cur_4.7.json'%EcN_ID)

In [238]:
#Establish a definition that initializes models to an in silico representation of gut microbiome media
def M1(model):
    
    medium = model.medium
    
    # Set exchange to zero
    for rxn in medium:
        medium[rxn] = 0.0 # Remove all current media components

    # Read medias - FROM AGORA
    media_db = pd.read_table("../tables/media_db.txt", comment="#")
    media_db["compound"] = [cpd.strip() for cpd in media_db["compound"]]
    media_db = media_db[["medium", "compound"]].groupby("medium").agg(lambda x: list(x)).to_dict()['compound']


    # Add compounds to media composition
    for compound in media_db['M1']: # M1 > Gut microbium media, GMM
        try:
            model.reactions.get_by_id('EX_%s_e'%compound)
            medium['EX_%s_e'%compound]= 1000
        except:
#             print(compound, 'is missing in the model')
            pass
    
    medium['EX_o2_e']= 1000 # Make aerobic, so we can analyse the differences between aerobic and anaerobic
        
    # Update medium
    model.medium = medium
    
    return model

In [19]:
# Get growth rate for each single gene knockout
with EcN_model:
    M1(EcN_model)
    wt_growth_ae = EcN_model.optimize().objective_value
    ae_results = single_gene_deletion(EcN_model) 
    
with EcN_model:
    M1(EcN_model)
    
    # Make anaerobic
    medium = EcN_model.medium
    medium['EX_o2_e']= 0.0
    EcN_model.medium = medium
    
    wt_growth_an = EcN_model.optimize().objective_value
    an_results = single_gene_deletion(EcN_model)

ae_results.head()

3mb is missing in the model
cbl2 is missing in the model
fol is missing in the model
hco3 is missing in the model
mndn is missing in the model
ribflv is missing in the model
3mb is missing in the model
cbl2 is missing in the model
fol is missing in the model
hco3 is missing in the model
mndn is missing in the model
ribflv is missing in the model


,ids,growth,status
0,{CIW80_20120},40.264903,optimal
1,{CIW80_07000},40.264903,optimal
2,{CIW80_10650},40.264903,optimal
3,{CIW80_00135},40.264903,optimal
4,{CIW80_22850},40.264903,optimal


In [20]:
# Get binary representation of gene essentiality
ae_results['binary_ae']= ae_results['growth'].map(lambda x: 0 if x < (0.05 * wt_growth_ae) else 1)
an_results['binary_an']= an_results['growth'].map(lambda x: 0 if x < (0.05 * wt_growth_an) else 1)

In [21]:
# Check whether there are genes for which the status is not "optimal"
ae_results[ae_results.status != 'optimal']

,ids,growth,status,binary_ae


In [22]:
# Check whether there are genes for which the status is not "optimal"
infeasible_an = an_results[an_results.status != 'optimal'].index.tolist()

# Assume infeasible result as growth deficient
for gene in infeasible_an:
    an_results.loc[gene, 'binary_an'] = 0
    
an_results[an_results.status != 'optimal']

,ids,growth,status,binary_an


In [23]:
# Unpack the index values from the frozenset to enable comparison to the ortho_matrices
for del_result in [ae_results, an_results]:
    for value in del_result.index:
        unpacked, = del_result.loc[value, 'ids']
        del_result.loc[value, 'gene'] = unpacked

    # Set the unpacked gene name as index
    del_result.set_index('gene', inplace=True)

In [24]:
# Remove the other two columsn
ae_results.drop(columns=['growth', 'status'], inplace=True)
an_results.drop(columns=['growth', 'status'], inplace=True)

# Merge the two dataframes and 
del_results = pd.merge(ae_results, an_results, left_index=True, right_index=True)

# Get an overview of the total number of essential genes
print('total =', len(del_results))
print('Aerobic essential =', len(del_results) - del_results['binary_ae'].sum())
print('Anaerobic essential =', len(del_results) - del_results['binary_an'].sum())

# Identify the genes that are only essential in one of the two conditions
del_results.drop(columns=['ids_x', 'ids_y'], inplace=True) # Drop the ids columns
del_results['diff'] = del_results['binary_ae'] - del_results['binary_an']
del_results[del_results['diff'] != 0]

total = 1533
Aerobic essential = 154
Anaerobic essential = 155


,binary_ae,binary_an,diff
gene,,,
CIW80_14315,1,0,1


In [25]:
del_results_ae = del_results[del_results['binary_ae'] == 0]
del_results_an = del_results[del_results['binary_an'] == 0]

# Save the resulting dataframe as .csv
del_results.to_csv('../tables/gene_ess_del_results.csv')

In [26]:
print(del_results.sum())

binary_ae    1379
binary_an    1378
diff            1
dtype: int64


In [27]:
# Find out which genes were essential only in anaerobic conditions
for gene in del_results[del_results['diff'] != 0].index:
    name = EcN_model.genes.get_by_id(gene).name
    print(name)

hemN


### Investigate hemN

In [13]:
EcN_model.genes.CIW80_14315

Gene identifier,CIW80_14315
Name,hemN
Memory address,0x0242c525ff08
Functional,True
In 1 reaction(s),CPPPGO2


In [14]:
import escher
from escher import Builder

In [15]:
builder = Builder(
    map_name='iJO1366.Central metabolism',
    model_name='iJO1366')

In [16]:
builder

Builder()

In [17]:
with EcN_model:
    M1(EcN_model)
    EcN_model.reactions.EX_o2_e.lower_bound=0
    solution = EcN_model.optimize()

3mb is missing in the model
cbl2 is missing in the model
fol is missing in the model
hco3 is missing in the model
mndn is missing in the model
ribflv is missing in the model


In [18]:
builder.reaction_data = solution.fluxes

In [19]:
EcN_model.genes.CIW80_14315

Gene identifier,CIW80_14315
Name,hemN
Memory address,0x0242c525ff08
Functional,True
In 1 reaction(s),CPPPGO2


In [20]:
EcN_model.genes.CIW80_06055

Gene identifier,CIW80_06055
Name,hemF
Memory address,0x0242c51ecd88
Functional,True
In 1 reaction(s),CPPPGO


In [21]:
EcN_model.reactions.CPPPGO2

Reaction identifier,CPPPGO2
Name,Oxygen Independent coproporphyrinogen-III oxidase
Memory address,0x0242c818da48
Stoichiometry,2.0 amet_c + cpppg3_c --> 2.0 co2_c + 2.0 dad_5_c + 2.0 met__L_c + pppg9_c 2.0 S-Adenosyl-L-methionine + Coproporphyrinogen III --> 2.0 CO2 CO2 + 2.0 5'-Deoxyadenosine + 2.0 L-Methionine + Protoporphyrinogen IX
GPR,CIW80_14315
Lower bound,0.0
Upper bound,1000.0


In [22]:
EcN_model.reactions.CPPPGO

Reaction identifier,CPPPGO
Name,Coproporphyrinogen oxidase (O2 required)
Memory address,0x0242c533b808
Stoichiometry,cpppg3_c + 2.0 h_c + o2_c --> 2.0 co2_c + 2.0 h2o_c + pppg9_c Coproporphyrinogen III + 2.0 H+ + O2 O2 --> 2.0 CO2 CO2 + 2.0 H2O H2O + Protoporphyrinogen IX
GPR,CIW80_06055
Lower bound,0.0
Upper bound,1000.0


The oxygen-dependency of hemF makes it unable to rescue grown anaerobically for an hemN knockout strain.

# II. Essential reactions EcN & subsystem

In [23]:
# Get growth rate for each single gene knockout
with EcN_model:
    M1(EcN_model)
    EcN_model.reactions.EX_o2_e.lower_bound=0 # Check essential reactions for anaerobic conditions
    wt_growth_an = EcN_model.optimize().objective_value
    an_rxn = single_reaction_deletion(EcN_model)

# Get binary representation of gene essentiality
an_rxn['binary']= an_rxn['growth'].map(lambda x: 0 if x < (0.05 * wt_growth_an) else 1)

# Check whether there are genes for which the status is not "optimal"
inf_rxn_an = an_rxn[an_rxn.status != 'optimal'].index.tolist()

# Assume infeasible result as growth deficient
for gene in inf_rxn_an:
    an_rxn.loc[gene, 'binary'] = 0
an_rxn[an_rxn.status != 'optimal']

3mb is missing in the model
cbl2 is missing in the model
fol is missing in the model
hco3 is missing in the model
mndn is missing in the model
ribflv is missing in the model


,ids,growth,status,binary


In [24]:
# Unpack the index values from the frozenset to enable comparison to the ortho_matrices
for value in an_rxn.index:
    unpacked, = an_rxn.loc[value, 'ids']
    an_rxn.loc[value, 'reaction'] = unpacked
    
# Set the unpacked gene name as index
an_rxn = an_rxn.set_index('reaction')
an_rxn.rename(columns = {'binary':'an_rxn_growth'}, inplace=True)

# Get the subsystem for all reactions
for rxn in an_rxn.index.tolist():
    an_rxn.loc[rxn, 'Subsystem'] = EcN_model.reactions.get_by_id(rxn).subsystem
    
print('total =', len(an_rxn))
print('Essential =', len(an_rxn) - an_rxn.an_rxn_growth.sum())
print('Non-essential =', an_rxn.an_rxn_growth.sum())

an_rxn

total = 3145
Essential = 221
Non-essential = 2924


,ids,growth,status,an_rxn_growth,Subsystem
reaction,,,,,
ACP1_FMN,{ACP1_FMN},40.260011,optimal,1,Cofactor and Prosthetic Group Biosynthesis
FEOXAMUtpp,{FEOXAMUtpp},40.260011,optimal,1,Inorganic Ion Transport and Metabolism
OHEDH,{OHEDH},40.260011,optimal,1,Tyrosine metabolism
AGPAT140,{AGPAT140},40.260011,optimal,1,Glycerophospholipid Metabolism
PGP181abcpp,{PGP181abcpp},40.260011,optimal,1,"Transport, Inner Membrane"
...,...,...,...,...,...
DARBabcpp,{DARBabcpp},40.260011,optimal,1,"Transport, Inner Membrane"
EX_csn_e,{EX_csn_e},40.260011,optimal,1,Extracellular exchange
EX_mepn_e,{EX_mepn_e},40.260011,optimal,1,Extracellular exchange


In [25]:
an_rxn[an_rxn['an_rxn_growth'] == 0 ]['Subsystem'].value_counts()

Cofactor and Prosthetic Group Biosynthesis            53
Purine and Pyrimidine Biosynthesis                    17
Cell Envelope Biosynthesis                            17
Tyrosine, Tryptophan, and Phenylalanine Metabolism    16
Valine, Leucine, and Isoleucine Metabolism            14
Extracellular exchange                                13
Arginine and Proline Metabolism                       12
Threonine and Lysine Metabolism                       12
Lipopolysaccharide Biosynthesis / Recycling           12
Methionine Metabolism                                  9
Transport, Outer Membrane                              9
Glycerophospholipid Metabolism                         8
Nucleotide Salvage Pathway                             5
Citric Acid Cycle                                      4
Inorganic Ion Transport and Metabolism                 3
Transport, Inner Membrane                              3
Alanine and Aspartate Metabolism                       2
Membrane Lipid Metabolism      

# VI. Double gene essentiality

In [ ]:
# Create a list of all non-essential genes
ae_non = del_results[del_results['binary_ae'] == 1].index.tolist()

# Run the double gene deletion with only the non-essential genes
with EcN_model:
    M1(EcN_model)
    double_ae = double_gene_deletion(EcN_model, ae_non, return_frame=True).round(4)
    
double_ae.head()

3mb is missing in the model
cbl2 is missing in the model
fol is missing in the model
hco3 is missing in the model
mndn is missing in the model
ribflv is missing in the model


In [ ]:
# Get binary representation of gene essentiality
double_ae['binary']= double_ae['growth'].map(lambda x: 0 if x < (0.05 * wt_growth_ae) else 1)

# Check whether there are genes for which the status is not "optimal"
inf_double = double_ae[double_ae.status != 'optimal'].index.tolist()

# Assume infeasible result as growth deficient
for gene in inf_double:
    double_ae.loc[gene, 'binary'] = 0
    
double_ae[double_ae.status != 'optimal']

In [ ]:
double_ess = double_ae[double_ae.status == 'optimal']
double_ess = double_ess[double_ess['binary'] == 0]
print(len(double_ess))
double_ess.head()

In [ ]:
# Save the resulting dataframe as .csv
double_ess.to_csv('../tables/gene_ess_double_ess.csv')

### Anaerobic

In [ ]:
# Create a list of all non-essential genes
an_non = del_results[del_results['binary_an'] == 1].index.tolist()

# Run the double gene deletion with only the non-essential genes
with EcN_model:
    M1(EcN_model)
    
    # Make anaerobic
    medium = EcN_model.medium
    medium['EX_o2_e']= 0.0
    EcN_model.medium = medium
    
    double_an = double_gene_deletion(EcN_model, an_non, return_frame=True).round(4)
    
double_an.head()

In [ ]:
# Get binary representation of gene essentiality
double_an['binary']= double_an['growth'].map(lambda x: 0 if x < (0.05 * wt_growth_an) else 1)

# Check whether there are genes for which the status is not "optimal"
inf_double_an = double_an[double_an.status != 'optimal'].index.tolist()

# Assume infeasible result as growth deficient
for gene in inf_double_an:
    double_an.loc[gene, 'binary'] = 0
    
double_an[double_an.status != 'optimal']

In [ ]:
double_ess_an = double_an[double_an.status == 'optimal']
double_ess_an = double_ess_an[double_ess_an['binary'] == 0]
print(len(double_ess_an))
double_ess_an.head()

In [ ]:
# Save the resulting dataframe as .csv
double_ess_an.to_csv('../tables/gene_ess_double_ess_an.csv')

In [22]:
# Get the names of the genes in the right format to compare
EcN_double = double_ess.copy()

for EcN_loc in EcN_double.index.tolist():
    EcN_id = EcN_double.loc[EcN_loc, 'ids']
    EcN_genes = []
    for gene in EcN_id:
        EcN_genes.append(str(gene))
    EcN_double.loc[EcN_loc, 'EcN'] = str(EcN_genes)

EcN_double_an = double_ess_an.copy()
for EcN_loc in EcN_double_an.index.tolist():
    EcN_id = EcN_double_an.loc[EcN_loc, 'ids']
    EcN_genes = []
    for gene in EcN_id:
        EcN_genes.append(str(gene))
    EcN_double_an.loc[EcN_loc, 'EcN'] = str(EcN_genes)

In [17]:
# Create two sets
set_ae = set(EcN_double['EcN'])
set_an = set(EcN_double_an['EcN'])

diff_ae = set_ae - set_an
diff_an = set_an - set_ae
similar = set.intersection(set_ae, set_an)

print(len(diff_ae), len(diff_an), len(similar))

1 0 153


In [60]:
def gene_names(gene_set):
    df = pd.DataFrame(columns=['EcN_id', 'EcN_id_1', 'EcN_id_2', 'EcN_name_1', 'EcN_name_2', 'EcN_names'])

    df['EcN_id'] = list(gene_set)
    for loc in df.index:
        
        #Split the name in two gene_ids
        name = df.loc[loc, 'EcN_id']
        name = name.strip("['").strip("]'").split("', '")
        df.loc[loc, 'EcN_id_1'] = name[0]
        df.loc[loc, 'EcN_id_2'] = name[1]
        
        # Identify the gene names
        df.loc[loc, 'EcN_name_1'] = EcN_model.genes.get_by_id(name[0]).name
        df.loc[loc, 'EcN_name_2'] = EcN_model.genes.get_by_id(name[1]).name
        name = sorted(name)
        df.loc[loc, 'EcN_names'] = str(EcN_model.genes.get_by_id(name[0]).name + ', ' + EcN_model.genes.get_by_id(name[1]).name)
        
    return df

In [78]:
# Get the names of the aerobic double essential genes
gene_overview_ae = gene_names(diff_ae)
gene_overview_ae

In [9]:
# Get the names of the anaerobic double essential genes
gene_overview_an = gene_names(diff_an)
gene_overview_an

,EcN_id,EcN_id_1,EcN_id_2,EcN_name_1,EcN_name_2,EcN_names


"hemF/hemN" is only found in aerobic conditions

##### Load the tables not to rerun the double_gene_deletion

In [56]:
# Load if needed
double_ess = pd.read_csv('../tables/gene_ess_double_ess.csv')
double_ess.set_index('Unnamed: 0', inplace=True)
double_ess_an = pd.read_csv('../tables/gene_ess_double_ess_an.csv')
double_ess_an.set_index('Unnamed: 0', inplace=True)

In [57]:
# get usable gene names
for EcN_loc in double_ess.index.tolist():
    gene_list = double_ess.loc[EcN_loc, 'ids'].split("', '")
    gene_1 = gene_list[0][2:]
    gene_2 = gene_list[1][:-2]
    double_ess.loc[EcN_loc, 'genes'] = gene_1 + ', ' + gene_2
    
for EcN_loc in double_ess_an.index.tolist():
    gene_list = double_ess_an.loc[EcN_loc, 'ids'].split("', '")
    gene_1 = gene_list[0][2:]
    gene_2 = gene_list[1][:-2]
    double_ess_an.loc[EcN_loc, 'genes_an'] = gene_1 + ', ' + gene_2
    
# Create two sets
set_ae = set(double_ess['genes'])
set_an = set(double_ess_an['genes_an'])

diff_ae = set_ae - set_an
diff_an = set_an - set_ae
similar = set.intersection(set_ae, set_an)

print('Total number of synthetic lethals aerobically:', len(set_ae), 'and anaerobically:', len(set_an))
print('The difference is:', len(diff_ae), len(diff_an), len(similar))
diff_ae

Total number of synthetic lethals aerobically: 154 and anaerobically: 153
The difference is: 1 0 153


{'CIW80_14315, CIW80_06055'}

### Check differences

In [ ]:
# Get the reactions
EcN_model.genes.CIW80_06055 #hemF
EcN_model.genes.CIW80_14315 #hemN

In [ ]:
# Check the double lethal only for aerobic
with EcN_model:
    M1(EcN_model)
    EcN_model.reactions.EX_o2_e.lower_bound=0
    EcN_model.remove_reactions(['CPPPGO2', 'CPPPGO'])
    print(EcN_model.slim_optimize())

This deletion is lethal in both cases, but was not included for anaerobic, as the deletion of only CIW80_14315 (hemN/CPPPGO2) is already lethal in anaerobic conditions

# VII. Gene comparison EcN & MG1655 model

In [8]:
# Load iML1515 model
k12_model = cobra.io.load_json_model('../data/models/iML1515.json')

# Get growth rate for each single gene knockout
with k12_model:
    M1(k12_model)
    k12_model.reactions.EX_o2_e.lower_bound=0
    del_result_k12 = single_gene_deletion(k12_model)

3mb is missing in the model
4abz is missing in the model
cbl2 is missing in the model
fol is missing in the model
hco3 is missing in the model
mndn is missing in the model
ribflv is missing in the model


In [9]:
# Get a binary representation of the gene essentiality
k12_growth = k12_model.optimize().objective_value

del_result_k12['binary']= del_result_k12['growth'].map(lambda x: 0 if x < (0.05 * k12_growth) else 1)

In [10]:
# Check whether there are genes for which the status is not "optimal"
del_result_k12[del_result_k12.status != 'optimal']

,ids,growth,status,binary


In [11]:
# Unpack the index values from the frozenset to enable comparison to the ortho_matrices
for value in del_result_k12.index:
    unpacked, = del_result_k12.loc[value, 'ids']
    del_result_k12.loc[value, 'gene'] = unpacked
                                   

# Set the unpacked gene name as index
del_result_k12 = del_result_k12.set_index('gene')
del_result_k12.rename(columns = {'binary':'K12_growth'}, inplace=True)
del_result_k12.drop(columns=['ids', 'growth', 'status'], inplace=True)
del_result_k12.head()

,K12_growth
gene,
b0432,1
b1584,1
b0420,0
b0311,1
b4213,1


In [28]:
# Get the ortho matrix of EcN vs. the reference strains
orthoIDs_matrix = pd.read_csv('../tables/orthoIDs_matrix.csv')
orthoIDs_matrix.set_index('Unnamed: 0', inplace=True)
geneIDs_sub = orthoIDs_matrix.loc[:, 'U00096'] # Get the gene ID orthologs for MG1655

# Merge the ortho_matrix with the EcN gene essentiality matrix
geneIDs_ess = pd.merge(geneIDs_sub, del_results, left_index = True, right_index = True)
geneIDs_ess.head()

,U00096,binary_ae,binary_an,diff
CIW80_00015,b1415,1,1,0
CIW80_00090,b1430,1,1,0
CIW80_00125,b1439,1,1,0
CIW80_00130,b1440,1,1,0
CIW80_00135,b1441,1,1,0


In [29]:
# Combine with previously generated EcN essentiality data merged with K12 gene IDs
ess_comp = pd.merge(geneIDs_ess, del_result_k12, left_on ='U00096', right_index =True)

# Find the differences
ess_comp['difference'] = ess_comp['binary_an'] - ess_comp['K12_growth']
ess_comp[ess_comp['difference'] != 0] 

,U00096,binary_ae,binary_an,diff,K12_growth,difference
CIW80_09665,b3040,1,1,0,0,1
CIW80_18070,b0073,0,0,0,1,-1
CIW80_18385,b0126,0,0,0,1,-1


In [30]:
# Save the resulting dataframe as .csv
ess_comp_k12 = ess_comp#[ess_comp['K12_growth'] == 0]
ess_comp_k12.to_csv('../tables/gene_ess_comp_k12.csv')

# Get the number of shared genes
len(ess_comp[ess_comp['U00096'].str.contains('b')])

1386

In [17]:
for gene in ess_comp[ess_comp['difference'] != 0]['U00096']:
    name = k12_model.genes.get_by_id(gene).name
    print(name)

zupT
leuB
can


In [18]:
for gene in ess_comp[ess_comp['difference'] != 0].index:
    name = EcN_model.genes.get_by_id(gene).name
    print(name)

zupT
leuB
can


## K12 ess
### ZupT
heavy metal divalent cation transporter ZupT
https://ecocyc.org/gene?orgid=ECOLI&id=EG11167

### proB (corrected)
glutamate 5-kinase
https://ecocyc.org/gene?orgid=ECOLI&id=EG10768

### proA (corrected)
glutamate-5-semialdehyde dehydrogenase
https://ecocyc.org/gene?orgid=ECOLI&id=EG10767

## EcN ess
### argI/argF (corrected)
ornithine carbamoyltransferase
https://ecocyc.org/gene?orgid=ECOLI&id=EG10069

### leuB
3-isopropylmalate dehydrogenase
https://ecocyc.org/gene?orgid=ECOLI&id=EG11577

### can
carbonic anhydrase 2
https://ecocyc.org/gene?orgid=ECOLI&id=EG12319

In [19]:
## zupT
# EcN_model.genes.CIW80_09665
# EcN_model.reactions.COBALT2tpp
# EcN_model.metabolites.cobalt2_c
# EcN_model.reactions.COBALT2t3pp

# EcN_model.reactions.CU2tpp
# EcN_model.metabolites.cu2_p 
# EcN_model.reactions.CU2abcpp

## proA/B
# EcN_model.genes.CIW80_19485
# EcN_model.genes.CIW80_19490
# EcN_model.reactions.G5SD
# EcN_model.metabolites.glu5sa_c 
# EcN_model.reactions.NACODA
# EcN_model.reactions.ACODA
# EcN_model.metabolites.orn_c

## argI/argF
# EcN_model.genes.CIW80_16625
# EcN_model.reactions.OCBT

## leuB
# EcN_model.genes.CIW80_18070
# EcN_model.reactions.IPMD

# can
# EcN_model.genes.CIW80_18385
# EcN_model.reactions.HCO3E

In [20]:
# Check effect of specific reactions encoded by these genes
with k12_model:
    k12_model.remove_reactions(['OCBT'])
    growth = k12_model.slim_optimize()
    print(growth)

2.5072183489465458e-17
